In [1]:
sc

In [2]:
import csv

nypd_collisions = sc.textFile('nypd_carl2_part0.csv')
weather_data = sc.textFile('processed_2011-2017_final.csv')

In [3]:
list(enumerate(nypd_collisions.first().split(',')))

[(0, u'BOROUGH'),
 (1, u'CONTRIBUTING_FACTOR_VEHICLE_1'),
 (2, u'CONTRIBUTING_FACTOR_VEHICLE_2'),
 (3, u'CONTRIBUTING_FACTOR_VEHICLE_3'),
 (4, u'CONTRIBUTING_FACTOR_VEHICLE_4'),
 (5, u'CONTRIBUTING_FACTOR_VEHICLE_5'),
 (6, u'CROSS_STREET_NAME'),
 (7, u'DATETIME'),
 (8, u'LATITUDE'),
 (9, u'LOCATION'),
 (10, u'LONGITUDE'),
 (11, u'NUMBER_OF_CYCLIST_INJURED'),
 (12, u'NUMBER_OF_CYCLIST_KILLED'),
 (13, u'NUMBER_OF_MOTORIST_INJURED'),
 (14, u'NUMBER_OF_MOTORIST_KILLED'),
 (15, u'NUMBER_OF_PEDESTRIANS_INJURED'),
 (16, u'NUMBER_OF_PEDESTRIANS_KILLED'),
 (17, u'NUMBER_OF_PERSONS_INJURED'),
 (18, u'NUMBER_OF_PERSONS_KILLED'),
 (19, u'OFF_STREET_NAME'),
 (20, u'ON_STREET_NAME'),
 (21, u'UNIQUE_KEY'),
 (22, u'VEHICLE_TYPE_CODE_1'),
 (23, u'VEHICLE_TYPE_CODE_2'),
 (24, u'VEHICLE_TYPE_CODE_3'),
 (25, u'VEHICLE_TYPE_CODE_4'),
 (26, u'VEHICLE_TYPE_CODE_5'),
 (27, u'ZIP_CODE')]

In [4]:
list(enumerate(weather_data.first().split(',')))

[(0, u''),
 (1, u'Conditions'),
 (2, u'DateUTC'),
 (3, u'DatetimeEDT'),
 (4, u'Dew_PointF'),
 (5, u'Events'),
 (6, u'Foggy'),
 (7, u'Freezing'),
 (8, u'Gust_SpeedMPH'),
 (9, u'Humidity'),
 (10, u'PrecipitationIn'),
 (11, u'Rain'),
 (12, u'Sea_Level_PressureIn'),
 (13, u'Snow'),
 (14, u'TemperatureF'),
 (15, u'TimeEDT'),
 (16, u'VisibilityMPH'),
 (17, u'WindDirDegrees'),
 (18, u'Wind_Direction'),
 (19, u'Wind_SpeedMPH'),
 (20, u'Windy')]

In [30]:
import csv
import datetime
def map_weather_hours(idx, part):
    if idx == 0:
        part.next()
    for p in csv.reader(part):
        datehour= p[3].split(':')[0]
        yield (datehour, p)

weather_data_rdd = weather_data.mapPartitionsWithIndex(map_weather_hours)
sorted(weather_data_rdd.groupByKey().mapValues(list).collect())

[('2011-01-01 00',
  [['0',
    'Clear',
    '2011-01-01 05:51:00',
    '2011-01-01 00:51:00',
    '28.9',
    '',
    '1',
    '0',
    '-',
    '60',
    'N/A',
    '0',
    '30.13',
    '0',
    '42.1',
    '12:51 AM',
    '7.0',
    '240',
    'WSW',
    '5.8',
    '0']]),
 ('2011-01-01 01',
  [['1',
    'Clear',
    '2011-01-01 06:51:00',
    '2011-01-01 01:51:00',
    '28.9',
    '',
    '1',
    '0',
    '-',
    '58',
    'N/A',
    '0',
    '30.12',
    '0',
    '43.0',
    '1:51 AM',
    '7.0',
    '240',
    'WSW',
    '8.1',
    '0']]),
 ('2011-01-01 02',
  [['2',
    'Clear',
    '2011-01-01 07:51:00',
    '2011-01-01 02:51:00',
    '28.9',
    '',
    '1',
    '0',
    '-',
    '60',
    'N/A',
    '0',
    '30.12',
    '0',
    '42.1',
    '2:51 AM',
    '7.0',
    '260',
    'West',
    '4.6',
    '0']]),
 ('2011-01-01 03',
  [['3',
    'Clear',
    '2011-01-01 08:51:00',
    '2011-01-01 03:51:00',
    '30.0',
    '',
    '1',
    '0',
    '-',
    '65',
    'N/A',
    

In [35]:
import csv
from dateutil.parser import parse

def map_collision_hours(idx, part):
    if idx == 0:
        part.next()
    for p in csv.reader(part):
        datehour = str(parse(p[7])).split(':')[0]
        yield (datehour, p)

nypd_collisions_rdd = nypd_collisions.mapPartitionsWithIndex(map_collision_hours)
nypd_collisions_rdd.take(5)

[('2017-04-16 00',
  ['BRONX',
   'Turning Improperly',
   'Unspecified',
   '',
   '',
   '',
   'BRONX RIVER AVENUE',
   '04/16/2017 0:00:00',
   '40.831287',
   '(40.831287, -73.88173)',
   '-73.88173',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '',
   'EAST 172 STREET',
   '3651861',
   'PASSENGER VEHICLE',
   'PICK-UP TRUCK',
   '',
   '',
   '',
   '10472']),
 ('2017-04-16 00',
  ['BROOKLYN',
   'Turning Improperly',
   'Unspecified',
   '',
   '',
   '',
   'SMITH STREET',
   '04/16/2017 0:00:00',
   '40.686726',
   '(40.686726, -73.99047)',
   '-73.99047',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '',
   'BERGEN STREET',
   '3651519',
   'TAXI',
   'PASSENGER VEHICLE',
   '',
   '',
   '',
   '11201']),
 ('2017-04-16 00',
  ['BROOKLYN',
   'Unspecified',
   'Unspecified',
   '',
   '',
   '',
   '',
   '04/16/2017 0:00:00',
   '40.70256',
   '(40.70256, -73.99114)',
   '-73.99114',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
